## DPDD status

In [1]:
import os
import numpy as np
#%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3
import lsst.daf.persistence as dafPersist
import lsst.afw.display as afwDisplay
import lsst.geom
afwDisplay.setDefaultBackend('matplotlib')
import matplotlib.ticker as plticker
from astropy.visualization import (ZScaleInterval, SqrtStretch, ImageNormalize)

This notebook gives me DIASource and DIAObject dataframes to futz with

In [2]:
!ls -lth /project/mrawls/hits2015/rerun/ | head

total 0
drwxrwsr-x 89 52919 lsst_users 4.0K Jan 30 05:19 cw_2020_01
drwxrwsr-x 89 52919 lsst_users 4.0K Jan  2 07:19 cw_2019_12v2
drwxrwsr-x 89 52919 lsst_users 4.0K Dec 20 09:47 cw_2019_12
drwxrwsr-x 89 52919 lsst_users 4.0K Dec 13 22:41 coaddtest7
drwxrwsr-x 89 52919 lsst_users 4.0K Dec 13 03:59 coaddtest8
drwxrwsr-x 89 52919 lsst_users 4.0K Dec 13 01:03 coaddtest9
drwxrwsr-x 89 52919 lsst_users 4.0K Nov 28 02:31 coaddtest6
drwxrwsr-x 89 52919 lsst_users 4.0K Nov 27 23:46 coaddtest5
drwxrwsr-x 89 52919 lsst_users 4.0K Nov 27 01:37 coaddtest4


In [3]:
# Meredith's standard processing location
REPO = '/project/mrawls/hits2015/rerun/cw_2020_01' 

In [4]:
connection = sqlite3.connect(f'{REPO}/association.db')

In [5]:
pd.read_sql_query('select count(diaSourceId) from DiaSource;', connection)

,count(diaSourceId)
0,634915


Take a look at the DiaSource table:

In [6]:
dia_sources = pd.read_sql_query('select * from DiaSource LIMIT 30;', connection)

In [7]:
dia_sources.head()

,diaSourceId,ccdVisitId,diaObjectId,ssObjectId,parentDiaSourceId,prv_procOrder,ssObjectReassocTime,midPointTai,ra,raErr,...,ixxPSF,iyyPSF,ixyPSF,extendedness,spuriousness,flags,pixelId,filterName,filterId,isDipole
0,176486665823453487,41091504,176486665823453487,0,0,0,None,57070.141866,149.505905,None,...,4.532675,3.881223,0.207817,None,None,25299404,0,g,1,0
1,176486665823453488,41091504,176486665823453488,0,0,0,None,57070.141866,149.506049,None,...,4.532675,3.881223,0.207817,None,None,25299404,0,g,1,0
2,176486665823453489,41091504,176486665823453489,0,0,0,None,57070.141866,149.511464,None,...,4.532675,3.881223,0.207817,None,None,25166152,0,g,1,0
3,176486665823453490,41091504,176486665823453490,0,0,0,None,57070.141866,149.511534,None,...,4.532675,3.881223,0.207817,None,None,25168328,0,g,1,0
4,176486665823453491,41091504,176486665823453491,0,0,0,None,57070.141866,149.512223,None,...,4.532675,3.881223,0.207817,None,None,41943040,0,g,1,1


In [33]:
dia_sources[dia_sources.dipMeanFlux == dia_sources.dipMeanFlux.max()]

,diaSourceId,ccdVisitId,diaObjectId,ssObjectId,parentDiaSourceId,prv_procOrder,ssObjectReassocTime,midPointTai,ra,raErr,...,ixxPSF,iyyPSF,ixyPSF,extendedness,spuriousness,flags,pixelId,filterName,filterId,isDipole
28,176486854802014339,41091548,176486854802014339,0,0,0,None,57070.141866,150.281299,None,...,3.267747,2.995904,-0.24153,None,None,1848,0,g,1,1


In [34]:
print(dia_sources.loc[28].to_string())

diaSourceId                    176486854802014339
ccdVisitId                               41091548
diaObjectId                    176486854802014339
ssObjectId                                      0
parentDiaSourceId                               0
prv_procOrder                                   0
ssObjectReassocTime                          None
midPointTai                               57070.1
ra                                        150.281
raErr                                        None
decl                                      1.85005
declErr                                      None
ra_decl_Cov                                  None
x                                         355.977
xErr                                         None
y                                         2145.26
yErr                                         None
x_y_Cov                                      None
apFlux                                     292976
apFluxErr                                 1954.74


In [21]:
dia_objects = pd.read_sql_query('select * from DiaObject LIMIT 30;', connection)

In [22]:
dia_objects.head()

,diaObjectId,validityStart,validityEnd,ra,raErr,decl,declErr,ra_decl_Cov,radecTai,pmRa,...,yPSFluxPercentile75,yPSFluxPercentile95,yPSFluxMin,yPSFluxMax,yPSFluxStetsonJ,yPSFluxLinearSlope,yPSFluxLinearIntercept,yPSFluxMaxSlope,yPSFluxErrMean,parent
0,176486854802014311,2015-02-17 03:24:17.200177,2015-02-17 05:27:34.070729,150.127240,0.0,1.770000,0.0,0.0,57070.141866,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,176486854802014312,2015-02-17 03:24:17.200177,None,150.127927,0.0,1.814163,0.0,0.0,57070.141866,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,176486854802014313,2015-02-17 03:24:17.200177,2015-02-17 05:27:34.070729,150.131793,0.0,1.799394,0.0,0.0,57070.141866,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,176486854802014314,2015-02-17 03:24:17.200177,2015-02-17 08:14:06.918801,150.133856,0.0,1.872403,0.0,0.0,57070.141866,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,176486854802014315,2015-02-17 03:24:17.200177,2015-02-17 05:27:34.070729,150.139030,0.0,1.818425,0.0,0.0,57070.141866,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [35]:
print(dia_objects.iloc[1].to_string())

diaObjectId                       176486854802014312
validityStart             2015-02-17 03:24:17.200177
validityEnd                                     None
ra                                           150.128
raErr                                              0
decl                                         1.81416
declErr                                            0
ra_decl_Cov                                        0
radecTai                                     57070.1
pmRa                                               0
pmRaErr                                            0
pmDecl                                             0
pmDeclErr                                          0
parallax                                           0
parallaxErr                                        0
pmRa_pmDecl_Cov                                    0
pmRa_parallax_Cov                                  0
pmDecl_parallax_Cov                                0
pmParallaxLnL                                 

In [25]:
dia_forced_source = pd.read_sql_query('select * from DiaForcedSource LIMIT 30;', connection)

In [27]:
dia_forced_source.head()

,diaObjectId,ccdVisitId,psFlux,psFluxErr,x,y,flags,diaForcedSourceId,totFlux,totFluxErr,midPointTai
0,176486854802014311,41091548,-1735.581655,316.551368,1450.913330,42.957546,0,88243427401007105,1.962407e+04,308.022661,57070.141866
1,176486854802014312,41091548,129696.843322,508.534728,846.933594,51.992020,0,88243427401007106,1.283730e+05,501.265191,57070.141866
2,176486854802014313,41091548,-4243.339949,283.877513,1048.869995,104.869934,0,88243427401007107,9.419917e+03,277.774383,57070.141866
3,176486854802014314,41091548,4492.142436,714.369391,50.728531,132.500671,0,88243427401007108,1.586398e+02,238.190817,57070.141866
4,176486854802014315,41091548,181503.723202,1743.550645,788.611235,203.495349,0,88243427401007109,1.812390e+06,1756.415356,57070.141866


In [28]:
dia_forced_source.iloc[0]

diaObjectId          1.764869e+17
ccdVisitId           4.109155e+07
psFlux              -1.735582e+03
psFluxErr            3.165514e+02
x                    1.450913e+03
y                    4.295755e+01
flags                0.000000e+00
diaForcedSourceId    8.824343e+16
totFlux              1.962407e+04
totFluxErr           3.080227e+02
midPointTai          5.707014e+04
Name: 0, dtype: float64

In [39]:
currentFluxMask = dia_sources["filterId"] == 1

In [33]:
fluxes = dia_sources.loc[:, "psFlux"].replace([None], np.nan)
fluxErrors = dia_sources.loc[:, "psFluxErr"].replace([None], np.nan)

In [34]:
noNanMask = np.logical_and(np.isfinite(fluxes), np.isfinite(fluxErrors))

In [35]:
noNanMask

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
20    True
21    True
22    True
23    True
24    True
25    True
26    True
27    True
28    True
29    True
Name: psFlux, dtype: bool

In [40]:
dia_sources.loc[currentFluxMask & noNanMask, "midPointTai"]

0     57070.141866
1     57070.141866
2     57070.141866
3     57070.141866
4     57070.141866
5     57070.141866
6     57070.141866
7     57070.141866
8     57070.141866
9     57070.141866
10    57070.141866
11    57070.141866
12    57070.141866
13    57070.141866
14    57070.141866
15    57070.141866
16    57070.141866
17    57070.141866
18    57070.141866
19    57070.141866
20    57070.141866
21    57070.141866
22    57070.141866
23    57070.141866
24    57070.141866
25    57070.141866
26    57070.141866
27    57070.141866
28    57070.141866
29    57070.141866
Name: midPointTai, dtype: float64

In [42]:
pd.Series(False,index=dia_objects.index)

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
dtype: bool